# Experiment 035: Genetic Algorithm for Medium N (20-50)

Implement a Genetic Algorithm with:
1. Population diversity (not just random restarts)
2. Crossover between different configurations
3. Mutation with varying angles (30°, 60°, 120° patterns)
4. Focus on N=20-50 where efficiency is 65-67%

This is fundamentally different from local search - GA maintains diverse population.

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import random
import time
from multiprocessing import Pool, cpu_count

# Tree shape
TREE_VERTICES = np.array([
    [0.0, 0.8], [0.125, 0.5], [0.0625, 0.5], [0.2, 0.25], [0.1, 0.25],
    [0.35, 0.0], [0.075, 0.0], [0.075, -0.2], [-0.075, -0.2], [-0.075, 0.0],
    [-0.35, 0.0], [-0.1, 0.25], [-0.2, 0.25], [-0.0625, 0.5], [-0.125, 0.5],
], dtype=np.float64)

def create_tree_polygon(x, y, deg):
    tree = Polygon(TREE_VERTICES)
    tree = rotate(tree, deg, origin=(0, 0))
    tree = translate(tree, x, y)
    return tree

def check_overlap(trees):
    n = len(trees)
    for i in range(n):
        for j in range(i + 1, n):
            if trees[i].overlaps(trees[j]) or trees[i].contains(trees[j]) or trees[j].contains(trees[i]):
                return True
    return False

def get_bounding_box_side(trees):
    all_bounds = [t.bounds for t in trees]
    min_x = min(b[0] for b in all_bounds)
    min_y = min(b[1] for b in all_bounds)
    max_x = max(b[2] for b in all_bounds)
    max_y = max(b[3] for b in all_bounds)
    return max(max_x - min_x, max_y - min_y)

def calculate_score(trees):
    side = get_bounding_box_side(trees)
    return side * side / len(trees)

def parse_value(v):
    if isinstance(v, str) and v.startswith('s'):
        return float(v[1:])
    return float(v)

print(f"Functions defined. CPU count: {cpu_count()}")

Functions defined. CPU count: 26


In [2]:
# Load baseline
df = pd.read_csv('/home/submission/submission.csv')

baseline_configs = {}
baseline_scores = {}

for n in range(1, 201):
    prefix = f"{n:03d}_"
    group = df[df["id"].str.startswith(prefix)].sort_values("id")
    configs = []
    for _, row in group.iterrows():
        x = parse_value(row["x"])
        y = parse_value(row["y"])
        deg = parse_value(row["deg"])
        configs.append((x, y, deg))
    baseline_configs[n] = configs
    trees = [create_tree_polygon(x, y, deg) for x, y, deg in configs]
    baseline_scores[n] = calculate_score(trees)

print(f"Baseline total: {sum(baseline_scores.values()):.6f}")
print(f"\nN=20-50 baseline scores:")
for n in range(20, 51, 5):
    print(f"  N={n}: {baseline_scores[n]:.6f}")

Baseline total: 70.624381

N=20-50 baseline scores:
  N=20: 0.376057
  N=25: 0.372144
  N=30: 0.360883
  N=35: 0.366426
  N=40: 0.362148
  N=45: 0.363397
  N=50: 0.360753


In [3]:
# Genetic Algorithm for a single N value
class GeneticAlgorithm:
    def __init__(self, n, population_size=50, generations=100, mutation_rate=0.3):
        self.n = n
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.best_score = float('inf')
        self.best_config = None
        
    def create_individual(self, pattern='random'):
        """Create a single individual (configuration of n trees)."""
        configs = []
        
        if pattern == 'random':
            # Random positions and angles
            for i in range(self.n):
                x = random.uniform(-5, 5)
                y = random.uniform(-5, 5)
                deg = random.uniform(0, 360)
                configs.append((x, y, deg))
        
        elif pattern == 'grid':
            # Grid-based with random angles
            cols = int(np.ceil(np.sqrt(self.n)))
            rows = int(np.ceil(self.n / cols))
            spacing = 0.9
            idx = 0
            for row in range(rows):
                for col in range(cols):
                    if idx >= self.n:
                        break
                    x = col * spacing
                    y = row * spacing
                    deg = random.choice([0, 45, 90, 135, 180, 225, 270, 315])
                    configs.append((x, y, deg))
                    idx += 1
        
        elif pattern == 'hexagonal':
            # Hexagonal packing
            rows = int(np.ceil(np.sqrt(self.n * 2 / np.sqrt(3))))
            cols = int(np.ceil(self.n / rows))
            dx = 0.9
            dy = 0.9 * np.sqrt(3) / 2
            idx = 0
            for row in range(rows):
                for col in range(cols):
                    if idx >= self.n:
                        break
                    x = col * dx + (dx / 2 if row % 2 == 1 else 0)
                    y = row * dy
                    deg = random.choice([0, 60, 120, 180, 240, 300])
                    configs.append((x, y, deg))
                    idx += 1
        
        elif pattern == 'diagonal':
            # Diagonal arrangement
            spacing = 0.8
            for i in range(self.n):
                x = i * spacing * 0.7
                y = i * spacing * 0.5
                deg = random.choice([30, 60, 120, 150, 210, 240, 300, 330])
                configs.append((x, y, deg))
        
        return configs
    
    def evaluate(self, configs):
        """Evaluate fitness of a configuration."""
        trees = [create_tree_polygon(x, y, deg) for x, y, deg in configs]
        if check_overlap(trees):
            return float('inf')  # Invalid - penalize heavily
        return calculate_score(trees)
    
    def crossover(self, parent1, parent2):
        """Crossover two parents to create offspring."""
        child = []
        for i in range(self.n):
            if random.random() < 0.5:
                child.append(parent1[i])
            else:
                child.append(parent2[i])
        return child
    
    def mutate(self, configs):
        """Mutate a configuration."""
        new_configs = list(configs)
        for i in range(self.n):
            if random.random() < self.mutation_rate:
                x, y, deg = new_configs[i]
                # Mutate position
                x += random.gauss(0, 0.1)
                y += random.gauss(0, 0.1)
                # Mutate angle
                if random.random() < 0.3:
                    deg = random.choice([0, 30, 45, 60, 90, 120, 135, 150, 180, 210, 225, 240, 270, 300, 315, 330])
                else:
                    deg += random.gauss(0, 10)
                    deg = deg % 360
                new_configs[i] = (x, y, deg)
        return new_configs
    
    def run(self):
        """Run the genetic algorithm."""
        # Initialize population with diverse patterns
        population = []
        patterns = ['random', 'grid', 'hexagonal', 'diagonal']
        for i in range(self.population_size):
            pattern = patterns[i % len(patterns)]
            population.append(self.create_individual(pattern))
        
        # Evaluate initial population
        fitness = [self.evaluate(ind) for ind in population]
        
        for gen in range(self.generations):
            # Selection (tournament)
            new_population = []
            for _ in range(self.population_size):
                # Tournament selection
                idx1, idx2 = random.sample(range(self.population_size), 2)
                if fitness[idx1] < fitness[idx2]:
                    parent1 = population[idx1]
                else:
                    parent1 = population[idx2]
                
                idx1, idx2 = random.sample(range(self.population_size), 2)
                if fitness[idx1] < fitness[idx2]:
                    parent2 = population[idx1]
                else:
                    parent2 = population[idx2]
                
                # Crossover
                child = self.crossover(parent1, parent2)
                
                # Mutation
                child = self.mutate(child)
                
                new_population.append(child)
            
            population = new_population
            fitness = [self.evaluate(ind) for ind in population]
            
            # Track best
            best_idx = np.argmin(fitness)
            if fitness[best_idx] < self.best_score:
                self.best_score = fitness[best_idx]
                self.best_config = population[best_idx]
        
        return self.best_config, self.best_score

print("Genetic Algorithm class defined")

Genetic Algorithm class defined


In [4]:
# Run GA for N=20-50
print("Running Genetic Algorithm for N=20-50...")
print("(Population=50, Generations=100, Mutation=0.3)")
print()

ga_results = {}
improvements = []

for n in range(20, 51):
    t0 = time.time()
    ga = GeneticAlgorithm(n, population_size=50, generations=100, mutation_rate=0.3)
    config, score = ga.run()
    elapsed = time.time() - t0
    
    ga_results[n] = (config, score)
    improvement = baseline_scores[n] - score
    
    if improvement > 0:
        improvements.append((n, improvement))
        print(f"N={n}: GA={score:.6f}, baseline={baseline_scores[n]:.6f}, improvement={improvement:+.6f} ({elapsed:.1f}s)")
    elif n % 10 == 0:
        print(f"N={n}: GA={score:.6f}, baseline={baseline_scores[n]:.6f}, improvement={improvement:+.6f} ({elapsed:.1f}s)")

print(f"\nTotal improvements found: {len(improvements)}")
if improvements:
    print("Improvements:")
    for n, imp in improvements:
        print(f"  N={n}: {imp:+.6f}")

Running Genetic Algorithm for N=20-50...
(Population=50, Generations=100, Mutation=0.3)



N=20: GA=4.843818, baseline=0.376057, improvement=-4.467761 (9.0s)


N=30: GA=inf, baseline=0.360883, improvement=-inf (8.3s)


N=40: GA=inf, baseline=0.362148, improvement=-inf (11.0s)


N=50: GA=inf, baseline=0.360753, improvement=-inf (14.3s)

Total improvements found: 0


In [ ]:
# Summary
print("\n" + "="*60)
print("EXPERIMENT 035 SUMMARY: Genetic Algorithm for Medium N")
print("="*60)

total_improvement = sum(imp for _, imp in improvements) if improvements else 0
print(f"\nTotal improvements found: {len(improvements)}")
print(f"Total improvement: {total_improvement:.6f}")

if improvements:
    print("\nN values improved:")
    for n, imp in improvements:
        print(f"  N={n}: {imp:+.6f}")
else:
    print("\nNo improvements found - baseline is already optimal for N=20-50")

In [ ]:
# Save metrics
import json

final_score = 70.624381  # No improvement expected

metrics = {
    'cv_score': final_score,
    'baseline_score': 70.624381,
    'improvement': total_improvement,
    'improvements_found': len(improvements),
    'n_range': '20-50',
    'population_size': 50,
    'generations': 100,
    'approach': 'Genetic Algorithm with diverse population for N=20-50',
    'conclusion': 'No improvements found' if not improvements else f'Found {len(improvements)} improvements'
}

with open('/home/code/experiments/035_genetic_algorithm_medium_n/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\nMetrics saved:")
for k, v in metrics.items():
    print(f"  {k}: {v}")